## ETL del dataset `steam_reviews`

En esta jupyter notebook se desarrolla la extracción, transformación y carga (ETL) del conjunto de datos `steam_reviews.json.gz`.

### Descripción de Columnas en user_reviews.gz.json

| Columna     | Descripción                                   | Ejemplo                                                                      |
|-------------|-----------------------------------------------|------------------------------------------------------------------------------|
| user_id     | Identificador único de usuario                | [76561197970982479, evcentric, maplemage]                                   |
| user_url    | URL del perfil del usuario                    | [http://steamcommunity.com/id/evcentric]                                    |
| reviews     | Revisión del usuario en formato JSON          | {'funny': '', 'posted': 'Posted September 8, 2013.', 'last_edited': '', 'item_id': '227300', 'helpful': '0 of 1 people (0%) found this review helpful', 'recommend': True, 'review': "For a simple (it's actually not all that simple but it can be!) truck driving Simulator, it is quite a fun and relaxing game. Playing on simple (or easy?) its just the basic WASD keys for driving but (if you want) the game can be much harder and realistic with having to manually change gears, much harder turning, etc. And reversing in this game is a ♥♥♥♥♥, as I imagine it would be with an actual truck. Luckily, you don't have to reverse park it but you get extra points if you do cause it is bloody hard. But this is surprisingly a nice truck driving game and I had a bit of fun with it."} |


In [44]:
import gzip  # Para trabajar con archivos comprimidos en formato Gzip
import json  # Para trabajar con datos en formato JSON
import matplotlib.pyplot as plt  # Para crear gráficos y visualizaciones
import numpy as np  # Para operaciones matemáticas eficientes y manipulación de arrays
import pandas as pd  # Para el análisis y manipulación de datos en forma de DataFrames
import pyarrow as pa  # Herramientas para trabajar con datos en formato de flecha (Arrow)
import pyarrow.parquet as pq  # Para trabajar con archivos en formato Parquet
import seaborn as sns  # Biblioteca de visualización de datos basada en matplotlib
import ast # Proporciona funciones para analizar y manipular el árbol de sintaxis abstracta
from datetime import datetime, timedelta

In [3]:
filas_review = []  # creamos una lista vacía para ir agregando las filas del archivo json
with gzip.open("data/user_reviews.json.gz", 'rt', encoding='utf-8') as file:
    for line in file.readlines():
        filas_review.append(ast.literal_eval(line))

# Se convierte en dataframe
reviews = pd.DataFrame(filas_review)
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Ahora es necesario desanidar los registros que contienen listas

In [4]:
duplicados_columnas = reviews[reviews.duplicated(subset=['user_id'], keep=False)] # Verificamos si hay filas duplicadas en la columna id
duplicados_columnas

,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [13]:
# Se revisan tres usuarios  de forma aleatoria

user_id = '05041129'
user_reviews = duplicados_columnas[duplicados_columnas['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('=' * 100)
    
user_id = 'SuchGayMuchWow'
user_reviews = duplicados_columnas[duplicados_columnas['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('=' * 100)
    
user_id = '76561198076474887'
user_reviews = duplicados_columnas[duplicados_columnas['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('=' * 100)

This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
10/10 too many children with mikes.
10/10 too many children with mikes.
Excelente e cada vez mais bonito.Comprei o jogo pouco tempo depois de seu lançamento, e cheguei as minhas primeiras 24 horas jogadas em um piscar de olhos. Com jogabilidade simples, bela musica e arte o jogo me conquistou quase que de imediato.Devo admitir que após algumas horas de jogo, me senti frustrado com alguns bugs falhas na contrução do mundo que fui encontrando enquanto ia evoluindo ou após comessar um novo jogo. Porem, após a ultima atualização, o jogo prativcamen

In [14]:
reviews = reviews.drop_duplicates(subset='user_id', keep='first')
duplicados_columnas = reviews[reviews.duplicated(subset=['user_id'], keep=False)]
duplicados_columnas

,user_id,user_url,reviews


In [15]:
reviews.shape

(25485, 3)

Se revisa la columna review

In [16]:
reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

| Campo         | Descripción                                                                      |
|---------------|----------------------------------------------------------------------------------|
| user_id       | Identificador único para el usuario.                                             |
| user_url      | URL del perfil del usuario en streamcommunity.                                   |
| reviews       | Lista de diccionarios que contienen información sobre los reviews del usuario.  |
| - funny       | Indica si alguien puso emoticón de gracioso al review.                           |
| - posted      | Fecha de posteo del review en formato "Posted April 21, 2011".                    |
| - last_edited | Fecha de la última edición del review.                                           |
| - item_id     | Identificador único del item (juego).                                           |
| - helpful     | Estadística donde otros usuarios indican si fue útil la información.            |
| - recommend   | Booleano que indica si el usuario recomienda o no el juego.                      |
| - review      | Sentencia string con los comentarios sobre el juego.                             |


In [17]:
review_norm = pd.json_normalize(reviews['reviews'].dropna()) # Utilizamos el método json_normalize para normalizar los datos
review_norm.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Se pierde user id y user url

In [18]:
# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
review_norm = pd.concat([reviews[['user_id', 'user_url']], review_norm], axis=1)
review_norm.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [19]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
review_norm = pd.melt(review_norm, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
review_norm.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


In [20]:
review_norm[review_norm['user_id']=='evcentric']

,user_id,user_url,variable,reviews
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
25801,evcentric,http://steamcommunity.com/id/evcentric,1,"{'funny': '', 'posted': 'Posted December 4, 20..."
51600,evcentric,http://steamcommunity.com/id/evcentric,2,"{'funny': '', 'posted': 'Posted November 3, 20..."
77399,evcentric,http://steamcommunity.com/id/evcentric,3,"{'funny': '', 'posted': 'Posted October 15, 20..."
103198,evcentric,http://steamcommunity.com/id/evcentric,4,"{'funny': '', 'posted': 'Posted October 15, 20..."
128997,evcentric,http://steamcommunity.com/id/evcentric,5,"{'funny': '', 'posted': 'Posted October 15, 20..."
154796,evcentric,http://steamcommunity.com/id/evcentric,6,None
180595,evcentric,http://steamcommunity.com/id/evcentric,7,None
206394,evcentric,http://steamcommunity.com/id/evcentric,8,None


In [21]:
# Se eliminan las filas con valor None
review_norm = review_norm.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
review_norm[review_norm['user_id']=='evcentric']

,user_id,user_url,variable,reviews
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
25801,evcentric,http://steamcommunity.com/id/evcentric,1,"{'funny': '', 'posted': 'Posted December 4, 20..."
51600,evcentric,http://steamcommunity.com/id/evcentric,2,"{'funny': '', 'posted': 'Posted November 3, 20..."
77399,evcentric,http://steamcommunity.com/id/evcentric,3,"{'funny': '', 'posted': 'Posted October 15, 20..."
103198,evcentric,http://steamcommunity.com/id/evcentric,4,"{'funny': '', 'posted': 'Posted October 15, 20..."
128997,evcentric,http://steamcommunity.com/id/evcentric,5,"{'funny': '', 'posted': 'Posted October 15, 20..."


In [22]:
# Se separan por columnas cada una de las claves de 'reviews'
reviews = review_norm['reviews'].apply(pd.Series, dtype='object')
reviews =reviews.add_prefix('reviews_')
reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [23]:
# Se une con el 'user_id' y 'user_url'
reviews = pd.concat([review_norm[['user_id', 'user_url']], reviews], axis=1)
reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [24]:
reviews.replace('', None, inplace=True)
# Mostrar las primeras filas del DataFrame
reviews.head()


,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [35]:
reviews.info()
print('=' * 100)
print(reviews.isna().sum().sort_values(ascending= False)/len(reviews) * 100)

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              57397 non-null  object
 1   user_url             57397 non-null  object
 2   reviews_funny        7899 non-null   object
 3   reviews_posted       57397 non-null  object
 4   reviews_last_edited  5898 non-null   object
 5   reviews_item_id      57397 non-null  object
 6   reviews_helpful      57397 non-null  object
 7   reviews_recommend    57397 non-null  bool  
 8   reviews_review       57367 non-null  object
dtypes: bool(1), object(8)
memory usage: 4.0+ MB
reviews_last_edited    89.724202
reviews_funny          86.237957
reviews_review          0.052268
user_id                 0.000000
user_url                0.000000
reviews_posted          0.000000
reviews_item_id         0.000000
reviews_helpful         0.000000
reviews_recommend       0.000000
dtype: floa

In [ ]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited'
reviews = reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])

In [39]:
reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

## Transformación de la columna 'reviews_posted'

Se necesita que la fecha donde se hizo el posteo de la review este en formato `YYYY-MM-DD`, pero se encuentra como `Posted November 5, 2011.`. Por lo tanto, es necesario procesar la fecha y extraer los elementos relevantes. Se utilizará expresiones regulares para buscar y capturar los valores de año, mes y día dentro de la cadena de texto.

In [41]:
reviews['reviews_posted'] = reviews['reviews_posted'].str.extract(r'Posted ([\w\s\d,]+)') # Utilizamos regex para extraer la fecha
reviews['reviews_posted'] = reviews['reviews_posted'].replace({'Posted': ''}, regex=True) # Eliminamos la palabra Posted
reviews['reviews_posted'] = pd.to_datetime(reviews['reviews_posted'], errors='coerce') # Convertimos la columna reviews_posted a datetime
reviews["reviews_posted"].isnull().sum() # Contamos los valores nulos de la columna reviews_posted

9771

In [45]:
print(reviews["reviews_posted"].max()) # Buscamos la fecha maxima
print(reviews["reviews_posted"].min()) # Buscamos la fecha minima

# Definimos un rango temporal adecuado para el conjunto de datos
start_date = datetime(2010, 10, 16)
end_date = datetime(2015, 12, 31)

2015-12-31 00:00:00
2010-10-16 00:00:00


In [46]:
# Imputamos fechas aleatorias a las filas con valores nulos en 'posted'
reviews.loc[reviews['reviews_posted'].isnull(), 'reviews_posted'] = np.random.choice(pd.date_range(start_date, end_date), reviews['reviews_posted'].isnull().sum())

In [48]:
reviews["reviews_posted"].isnull().sum() # Revisamos nuevamente la cantidad de valores nulos de la columna posted

0

In [49]:
# Extraemos el año de la columna "release_date" y creamos una nueva columna llamada "release_year"
reviews["posted_year"] = reviews["reviews_posted"].dt.year 
reviews.head(2)

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014


In [51]:
reviews = reviews.drop('reviews_posted', axis=1)

In [54]:
reviews.isna().sum().sort_values(ascending= False)/len(reviews) * 100

reviews_review       0.052268
user_id              0.000000
user_url             0.000000
reviews_item_id      0.000000
reviews_helpful      0.000000
reviews_recommend    0.000000
posted_year          0.000000
dtype: float64

In [55]:
reviews = reviews.dropna(subset=['reviews_review'])

In [56]:
reviews.isna().sum().sort_values(ascending= False)/len(reviews) * 100

user_id              0.0
user_url             0.0
reviews_item_id      0.0
reviews_helpful      0.0
reviews_recommend    0.0
reviews_review       0.0
posted_year          0.0
dtype: float64

In [59]:
reviews.to_csv("data/02-user_reviews.csv", index=False, encoding='utf-8') # Exportamos el dataframe a csv

In [60]:
reviews = pd.read_csv("data/02-user_reviews.csv") # leemos el archivo csv

tabla = pa.Table.from_pandas(reviews) # convertimos el dataframe en una tabla
pq.write_table(tabla,"data/02-user_reviews.parquet") # guardamos la tabla en un archivo parquet